In [1]:
import findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Домашнее задание 2") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта 
#(в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [5]:
from pyspark.sql.types import *

In [6]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("total_cases", IntegerType(), True)
                    ])

In [7]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [8]:
df.select('iso_code', 'location', 'total_cases') \
.filter((df.date == '2020-03-31') & (df.continent.isNotNull())) \
.sort(df.total_cases.desc()) \
.show(15)

+--------+--------------+-----------+
|iso_code|      location|total_cases|
+--------+--------------+-----------+
|     USA| United States|   192301.0|
|     ITA|         Italy|   105792.0|
|     ESP|         Spain|    95923.0|
|     CHN|         China|    82279.0|
|     DEU|       Germany|    71808.0|
|     FRA|        France|    52278.0|
|     IRN|          Iran|    44605.0|
|     GBR|United Kingdom|    38815.0|
|     CHE|   Switzerland|    16605.0|
|     TUR|        Turkey|    13531.0|
|     BEL|       Belgium|    12775.0|
|     NLD|   Netherlands|    12667.0|
|     AUT|       Austria|    10180.0|
|     KOR|   South Korea|     9786.0|
|     CAN|        Canada|     8527.0|
+--------+--------------+-----------+
only showing top 15 rows



In [9]:
'''
Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном 
порядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)
'''

'\nTop 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном \nпорядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)\n'

In [10]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("total_cases", IntegerType(), True)
                    ])

In [11]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [12]:
df.select('iso_code', 'location', 'total_cases', 'date') \
.filter((df.date >= '2020-03-24') &  (df.date <= '2020-03-31') & (df.continent.isNotNull())) \
.createOrReplaceTempView("hmmm")

In [13]:
spark.sql("Select t2.date, t1.* " +
          "from (select iso_code, location, max(total_cases) mx_total_cases " +
          "from hmmm " +
          "group by iso_code, location) t1 left join hmmm t2 on t1.iso_code = t2.iso_code and t1.mx_total_cases = t2.total_cases " +
          "order by t1.mx_total_cases desc").show(15)

+----------+--------+--------------+--------------+
|      date|iso_code|      location|mx_total_cases|
+----------+--------+--------------+--------------+
|2020-03-31|     USA| United States|      192301.0|
|2020-03-31|     ITA|         Italy|      105792.0|
|2020-03-31|     ESP|         Spain|       95923.0|
|2020-03-31|     CHN|         China|       82279.0|
|2020-03-31|     DEU|       Germany|       71808.0|
|2020-03-31|     FRA|        France|       52278.0|
|2020-03-31|     IRN|          Iran|       44605.0|
|2020-03-31|     GBR|United Kingdom|       38815.0|
|2020-03-31|     CHE|   Switzerland|       16605.0|
|2020-03-31|     TUR|        Turkey|       13531.0|
|2020-03-31|     BEL|       Belgium|       12775.0|
|2020-03-31|     NLD|   Netherlands|       12667.0|
|2020-03-31|     AUT|       Austria|       10180.0|
|2020-03-31|     KOR|   South Korea|        9786.0|
|2020-03-31|     CAN|        Canada|        8527.0|
+----------+--------+--------------+--------------+
only showing

In [14]:
'''
Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, 
кол-во новых случаев сегодня, дельта)
'''


'\nПосчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. \n(например: в россии вчера было 9150 , сегодня 8763, итог: -387) \n(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, \nкол-во новых случаев сегодня, дельта)\n'

In [15]:
schema = StructType([StructField("iso_code", StringType(), False),
                     StructField("continent", StringType(), True),
                     StructField("location", StringType(), False),
                     StructField("date", TimestampType(), False),
                     StructField("new_cases", IntegerType(), True)
                    ])

In [16]:
df = (spark.read
          .option('header', True)
          .option('sep', ',')
          .option('inferSchema', True)
          .csv('Downloads/owid-covid-data.csv')
     )

In [17]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead  


In [18]:
windowSpec  = Window.partitionBy("iso_code").orderBy("date")

In [19]:
ddf = df.select('iso_code', 'location', 'date', 'new_cases') \
.filter((df.iso_code == 'RUS') & (df.date >= '2020-03-24') &  (df.date <= '2020-04-01')) \
.withColumn("lead",lead("new_cases",-1).over(windowSpec)).na.fill(value=0)

In [20]:
ddf.withColumn('Result', ( ddf['new_cases'] - ddf['lead'] ) ).show()

+--------+--------+----------+---------+-----+------+
|iso_code|location|      date|new_cases| lead|Result|
+--------+--------+----------+---------+-----+------+
|     RUS|  Russia|2020-03-24|     57.0|  0.0|  57.0|
|     RUS|  Russia|2020-03-25|    163.0| 57.0| 106.0|
|     RUS|  Russia|2020-03-26|    182.0|163.0|  19.0|
|     RUS|  Russia|2020-03-27|    196.0|182.0|  14.0|
|     RUS|  Russia|2020-03-28|    228.0|196.0|  32.0|
|     RUS|  Russia|2020-03-29|    270.0|228.0|  42.0|
|     RUS|  Russia|2020-03-30|    302.0|270.0|  32.0|
|     RUS|  Russia|2020-03-31|    501.0|302.0| 199.0|
|     RUS|  Russia|2020-04-01|    440.0|501.0| -61.0|
+--------+--------+----------+---------+-----+------+



In [21]:
sc.stop()

NameError: name 'sc' is not defined